In [1]:
import json
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import sqlite3
from datetime import datetime, timedelta

Retrieving all data from local SQL database

In [2]:
#connecting to sql database and storing data in 'reddit_data
sql_db = 'reddit_db.sqlite'
conn = sqlite3.connect(sql_db)
cur = conn.cursor()

#cur.execute('''SELECT NBA_submissions.date_created,NBA_submissions.submission_score,NBA_subs.sub_name,NBA_submissions.submission_title FROM NBA_submissions,NBA_subs WHERE NBA_submissions.team_id = NBA_subs.id''')
#reddit_data = pd.DataFrame(data=cur.fetchall(),columns=['timestamp','submission_score','subreddit','submission_title'])



query = '''SELECT NBA_submissions.date_created,NBA_submissions.sub_red_id,NBA_subs.team_name,NBA_submissions.submission_title,NBA_submissions.reddit_url FROM NBA_submissions,NBA_subs WHERE NBA_submissions.team_id = NBA_subs.id'''
columns_list=['timestamp','submission_redditID','subreddit','submission_title','submission_url']

cur.execute(query)
reddit_data = pd.DataFrame(data=cur.fetchall(),columns=columns_list)

reddit_data.head()

,timestamp,submission_redditID,subreddit,submission_title,submission_url
0,2011-12-28 04:32:27 UTC,nt6qa,ATL,Hawks slam the Nets 106-70 in New Jersey for t...,www.reddit.com/r/AtlantaHawks/comments/nt6qa/h...
1,2011-12-18 15:27:42 UTC,nhd05,ATL,This made me sad: Why many NBA stars bypass A...,www.reddit.com/r/AtlantaHawks/comments/nhd05/t...
2,2011-12-16 19:26:26 UTC,nfh80,ATL,Free scrimmage game tonight (12/16),www.reddit.com/r/AtlantaHawks/comments/nfh80/f...
3,2011-12-14 03:45:24 UTC,nbxlw,ATL,fuck tmac,www.reddit.com/r/AtlantaHawks/comments/nbxlw/f...
4,2011-12-11 21:44:52 UTC,n8ufg,ATL,"Ruck Sund says Hawks don’t want to pay tax, ma...",www.reddit.com/r/AtlantaHawks/comments/n8ufg/r...


In [3]:
#change timestamp column to datetime objects
reddit_data['timestamp'] = reddit_data['timestamp'].apply(lambda x: pd.to_datetime(x.split()[0] + ' ' + x.split()[1]))

In [4]:
###contains NBA season start and end dates for each seasons offseason, regular season and playoffs
###the end date is incremented by one day

season_dates = {'2013': {'offseason' : ('06/21/2013','11/28/2013'),'reg_season':('11/29/2013','04/16/2014'),'playoffs':('04/19/2014','06/15/2014')}
, 
'2014': {'offseason' : ('06/21/2014','11/28/2014'),'reg_season':('10/28/2014','04/15/2015'),'playoffs':('04/18/2015','06/16/2015')}
,
'2015': {'offseason' : ('06/17/2015','10/26/2015'),'reg_season':('10/27/2015','04/13/2016'),'playoffs':('04/16/2016','06/19/2016')} 
,
'2016' : {'offseason' : ('06/20/2016','10/24/2016'),'reg_season':('10/25/2016','04/12/2017'),'playoffs':('04/15/2017','06/12/2017')}
               }

In [5]:
###contains all subreddit names
### should equal 30 teams + 1 for the NBA

subreddit_list = list(reddit_data['subreddit'].unique())
print('# of subreddits:',len(subreddit_list))

# of subreddits: 31


Function for extracting significant days of submissions will be defined below

In [6]:
def significant_submissions(reddit_df):
    '''
    (pandas df) -> [list of tuples(date,top submission that day,submission id)]
    
    provided a df of reddit submissions for 1 particular subreddit over a given period of time.
    will assert that for 'subr_name' field of df that there is only 1 unique value
    
    
    from this dataframe group total submissions counts per hour and then get a list of
    statistically significant spikes in post per hour.
    From this list filter it to so it only has unique day dates.
    
    For each day date retrieve data on the top submission  for that day,including its submission id and title.
    '''
    
    tot_postshour_df = reddit_df.groupby(pd.TimeGrouper(freq='H'))['subreddit'].count()
    post_per_hour = tot_postshour_df.reset_index()['subreddit']
    post_per_hour = post_per_hour[post_per_hour!=0]
    
    std_level = post_per_hour.std()*3
    mean_level = post_per_hour.mean()
    threshhold = round(mean_level + std_level)
    
    
    top_significant_times = tot_postshour_df[tot_postshour_df>threshhold]
    top_significant_times = top_significant_times.reset_index()
    top_significant_times = top_significant_times.sort_values('subreddit',ascending=False)
    
    
    
    
    #results
    significant_times = []
    
    
    for time in top_significant_times.values:
        significant_times.append({'timestamp':time[0],'std_measure':time[1]/std_level})
    

    significant_results = []
    for significant_date in significant_times:
        
        #within hour
        start_time = significant_date['timestamp'].to_pydatetime()
        end_time = start_time + timedelta(hours=1)
        
        start_day = start_time.strftime('%Y-%m-%d')
        end_day = end_time.strftime('%Y-%m-%d')
        
        if start_day == end_day:
            #search for dataframe for this day, then use df.between_time for posts within that hour.
            #get sliced data that is either 1 day or 2 day
            day_data = reddit_df[start_day]
        else:
            day_data = reddit_df[start_day:end_day]
            
        df_results = day_data.between_time(start_time.time(),end_time.time())
        top_title = df_results.reset_index().sort_values('submission_redditID',ascending=False)
        
        
        #start_time = start_time.strftime('%Y-%m-%d %H:%M:%S')
        #end_time = end_time.strftime('%Y-%m-%d %H:%M:%S')
        
        
        #retriving submission with highest score
        #top_title = reddit_df[start_time:end_time].sort_values('submission_score',ascending=False)['submission_title']
        
        
        if len(top_title) > 0:
            
            significant_results.append({'timestamp':significant_date['timestamp'].strftime('%Y-%m-%d %H:%M:%S'),'std_measure':significant_date['std_measure'],'submission_title':top_title['submission_title'].iloc[0],'submission_redditID':top_title['submission_redditID'].iloc[0],'submission_url':top_title['submission_url'].iloc[0]})
            
        else:
            continue
    
    #changing timestamp from timestamp object to string
    for time in significant_times:
        time['timestamp'] = time['timestamp'].strftime('%Y-%m-%d %H:%M:%S')
    return significant_results
    

    
    
def df_periodslicer(reddit_df,start_date,end_date='',team=''):
    
    '''
    (pd.df,string,string,string) -> pd.df indexed by timestamp field
    
    
    
    '''
    
    
    if end_date == '':
        
        return_df = reddit_data.set_index('timestamp')[start_date]
    else:
        return_df = reddit_data.set_index('timestamp')[start_date:end_date]
    
    if len(team) < 1:
        return return_df
    else:
        return return_df[return_df['subreddit']==team]
    
    
    
    
    
    

In [7]:
highlights_json = {}
season_types = ['reg_season','playoffs','offseason']


for year in season_dates:
    year_val = year
    year_data = {}
    
    for team in subreddit_list:
        subreddit_name = team
        team_data = {}
        
        for season_type in season_types:
            
            start_date = season_dates[year][season_type][0]
            end_date = season_dates[year][season_type][1]
            team_df = df_periodslicer(reddit_data,start_date,end_date,team)
            team_data[season_type] = significant_submissions(team_df)
        
        
        year_data[team] = team_data
    
    highlights_json[year_val] = year_data

    
highlights_json

{'2013': {'ATL': {'offseason': [{'std_measure': 1.911962620829075,
     'submission_redditID': '1i1plo',
     'submission_title': 'Am I the only one that wants nothing to do with a Teauge for Ellis/Jennings deal?',
     'submission_url': 'www.reddit.com/r/AtlantaHawks/comments/1i1plo/am_i_the_only_one_that_wants_nothing_to_do_with_a/?ref=search_posts',
     'timestamp': '2013-07-11 00:00:00'}],
   'playoffs': [],
   'reg_season': []},
  'BKN': {'offseason': [], 'playoffs': [], 'reg_season': []},
  'BOS': {'offseason': [{'std_measure': 1.5198911557191577,
     'submission_redditID': '1gxse1',
     'submission_title': 'Josh Smith',
     'submission_url': 'www.reddit.com/r/bostonceltics/comments/1gxse1/josh_smith/?ref=search_posts',
     'timestamp': '2013-06-24 00:00:00'}],
   'playoffs': [],
   'reg_season': []},
  'CHA': {'offseason': [],
   'playoffs': [{'std_measure': 1.754742217921697,
     'submission_redditID': '262w9r',
     'submission_title': 'Who should the Hornets take with t

In [25]:
'''
for item in highlights_json['2015']['lakers']['reg_season']:
    print(item['top_submission_title'])
'''
import json

with open('significant_subs.json','w') as out:
    json.dump(highlights_json,out)


#highlights_json['2013']['atlantahawks']['reg_season'][0]